In [211]:
import numpy as np
from shapely.geometry import LineString, Polygon
import geopandas as gpd

In [ ]:
# Read in linestring object
file = '/Users/hyin/usgs_mendenhall/ffsimmer/fault-extrusion/1857-ex/1852-geometry.shp'

# extract file base name (without extension) for later use
filename = file.split('/')[-1].split('.')[0]
print(f"Processing fault file: {filename}")

zmax = 10.0 # maximum depth for extrusion (km)
dip_deg = 80 # dip angle in degrees

trace = gpd.read_file(file)

# Convert to local UTM
utm_crs = trace.to_crs(trace.estimate_utm_crs()).crs
trace = trace.to_crs(utm_crs)

## Extract the individual segments from each linestring
line = trace.geometry.values[0] # assumes there's only one linestring in the shapefile
coords = list(line.coords)
print(coords)

# ## Convert to local UTM coordinates for extrusion
# utm_crs = trace.to_crs(trace.estimate_utm_crs()).crs
# print(f"Using local UTM CRS: {utm_crs}")
# trace_utm = trace.to_crs(utm_crs)
# line_utm = trace_utm.geometry.values[0]
# coords_utm = list(line_utm.coords)

# print(coords_utm)
# line_utm = trace_utm.geometry.values[0]




Processing fault file: 1852-geometry
[(206346.62674671714, 3955604.7212927854), (283630.28951624973, 3868939.927131421), (305710.937752818, 3856382.922522148), (336097.6577532587, 3852774.9350797893), (397952.94981139095, 3826264.033660983), (453239.4895008896, 3792677.5227329694), (491360.593494343, 3770722.6906648255)]


In [213]:
def strike_and_dipdir(p0, p1):
    '''
    Calculate strike and dip direction from two points using the right-hand rule.
    '''
    dx = p1[0] - p0[0]
    dy = p1[1] - p0[1]
    strike = np.degrees(np.arctan2(dx, dy)) % 360
    dipdir = (strike + 90) % 360
    return strike, dipdir


In [214]:
segment_data = []   # Initialize a list of segments with their extrusion vectors

for i in range(len(coords) - 1):
    start = coords[i]
    end = coords[i + 1]

    print(f"Segment {i}: start={start}, end={end}")

    strike, dipdir = strike_and_dipdir(start, end)
    print(f"Strike: {strike:.2f} degrees")
    print(f"Dip Direction: {dipdir:.2f} degrees")

    # Convert the dip direction to a unit vector in the horizontal plane (east, north)
    dip_rad = np.radians(dipdir)
    dip_xy = np.array([np.sin(dip_rad), np.cos(dip_rad)]) # east, north
    dip_xy /= np.linalg.norm(dip_xy)    # normalize to unit vector to get dip direction unit vector
    print(f"Dip direction unit vector (dip_xy): {dip_xy}")
    L_h = zmax / np.tan(np.radians(dip_deg))
    print(f"Horizontal extrusion length (L_h): {L_h:.2f} km")
    
    V = np.array([
        dip_xy[0] * L_h*1000,
        dip_xy[1] * L_h*1000,
        -zmax*1000
    ])
    print(f"Extrusion vector (V): {V}\n")

    segment_data.append({
    "start": start,
    "end": end,
    "V": V
    })

# example of how to access the data for the first segment
segment_data[0]['start']        # each segment has 'start', 'end', and 'V' keys

Segment 0: start=(206346.62674671714, 3955604.7212927854), end=(283630.28951624973, 3868939.927131421)
Strike: 138.27 degrees
Dip Direction: 228.27 degrees
Dip direction unit vector (dip_xy): [-0.74634662 -0.66555746]
Horizontal extrusion length (L_h): 8.39 km
Extrusion vector (V): [ -6262.59169482  -5584.69017655 -10000.        ]

Segment 1: start=(283630.28951624973, 3868939.927131421), end=(305710.937752818, 3856382.922522148)
Strike: 119.63 degrees
Dip Direction: 209.63 degrees
Dip direction unit vector (dip_xy): [-0.49434214 -0.86926742]
Horizontal extrusion length (L_h): 8.39 km
Extrusion vector (V): [ -4148.02310395  -7294.01970338 -10000.        ]

Segment 2: start=(305710.937752818, 3856382.922522148), end=(336097.6577532587, 3852774.9350797893)
Strike: 96.77 degrees
Dip Direction: 186.77 degrees
Dip direction unit vector (dip_xy): [-0.11790744 -0.99302459]
Horizontal extrusion length (L_h): 8.39 km
Extrusion vector (V): [  -989.3608626   -8332.46567276 -10000.        ]

Segme

(206346.62674671714, 3955604.7212927854)

## Deal with the extrusion conflicts at depth
For each "vertex" (intersection between two line segments) we need to find a common point at depth where the two planes can intersect

In [215]:
coords[0]

(206346.62674671714, 3955604.7212927854)

In [216]:
P_bottom = {}

# Calculate the first vertex
P0 = np.array([coords[0][0], coords[0][1], 0.0])    # Make surfaace point 3D by adding z=0
print(f"P0:{P0}")
P_bottom[0] = P0 + segment_data[0]["V"]

# Calculate the middle vertices
for i in range(1, len(coords) - 1):
    Pi = np.array([*coords[i], 0.0])

    V_left  = segment_data[i - 1]["V"]
    V_right = segment_data[i]["V"]

    P_left  = Pi + V_left
    P_right = Pi + V_right

    P_bottom[i] = 0.5 * (P_left + P_right)

# Calculate the last vertex
Pn = np.array([coords[-1][0], coords[-1][1], 0.0])
P_bottom[len(coords)-1] = Pn + segment_data[-1]["V"]

P_bottom

P0:[ 206346.62674672 3955604.72129279       0.        ]


{0: array([ 200084.03505189, 3950020.03111623,  -10000.        ]),
 1: array([ 278424.98211686, 3862500.57219145,  -10000.        ]),
 2: array([ 303142.24576954, 3848569.67983408,  -10000.        ]),
 3: array([ 333950.21112648, 3844752.4646108 ,  -10000.        ]),
 4: array([ 394121.87913054, 3818822.10302073,  -10000.        ]),
 5: array([ 448967.3281093 , 3785456.17737226,  -10000.        ]),
 6: array([ 487172.8796819 , 3763451.38595871,  -10000.        ])}

# Build subfault polygons

In [217]:
polys = []

for i in range(len(coords) - 1):
    P0 = np.array(coords[i])
    P1 = np.array(coords[i + 1])

    B0 = P_bottom[i]
    B1 = P_bottom[i + 1]

    poly = Polygon([
        (P0[0], P0[1], 0.0),
        (P1[0], P1[1], 0.0),
        (B1[0], B1[1], B1[2]),
        (B0[0], B0[1], B0[2]),
        (P0[0], P0[1], 0.0)
    ])

    polys.append(poly)

polys

[<POLYGON Z ((206346.627 3955604.721 0, 283630.29 3868939.927 0, 278424.982 3...>,
 <POLYGON Z ((283630.29 3868939.927 0, 305710.938 3856382.923 0, 303142.246 3...>,
 <POLYGON Z ((305710.938 3856382.923 0, 336097.658 3852774.935 0, 333950.211 ...>,
 <POLYGON Z ((336097.658 3852774.935 0, 397952.95 3826264.034 0, 394121.879 3...>,
 <POLYGON Z ((397952.95 3826264.034 0, 453239.49 3792677.523 0, 448967.328 37...>,
 <POLYGON Z ((453239.49 3792677.523 0, 491360.593 3770722.691 0, 487172.88 37...>]

In [218]:
gdf_out = gpd.GeoDataFrame(
    geometry=polys,
    crs=utm_crs).to_crs("EPSG:4326")   # Convert back to WGS84 for output

gdf_out.to_file(f"{filename}_dip{dip_deg}_z{str(zmax)}km_extruded.geojson", driver="GeoJSON")


## Geojson to `fault.json`
Write fault.json file in the format ShakeMap expects.
- A single MultiPolygon 
- Coordinates in the order surface trace, trace at depth, back to starting point. 

```
P0 ── P1 ── P2 ── ... ── PN
│                         │
│                         │
PN' ── ... ── P2' ── P1' ── P0'
```

In [219]:
# Rearrange polys into a single multi-polygon fault surface

polys

[<POLYGON Z ((206346.627 3955604.721 0, 283630.29 3868939.927 0, 278424.982 3...>,
 <POLYGON Z ((283630.29 3868939.927 0, 305710.938 3856382.923 0, 303142.246 3...>,
 <POLYGON Z ((305710.938 3856382.923 0, 336097.658 3852774.935 0, 333950.211 ...>,
 <POLYGON Z ((336097.658 3852774.935 0, 397952.95 3826264.034 0, 394121.879 3...>,
 <POLYGON Z ((397952.95 3826264.034 0, 453239.49 3792677.523 0, 448967.328 37...>,
 <POLYGON Z ((453239.49 3792677.523 0, 491360.593 3770722.691 0, 487172.88 37...>]

In [220]:
# Build surface trace
surface_trace = [

]

# bottom_trace = [
#     (lon, lat, depth),
#     ...
# ]